In [1]:
import numpy as np
from pathlib import Path
from PIL import Image
from torch.utils.data import DataLoader, random_split
import torch
from torch import optim
import torch.nn as nn
from tqdm import tqdm
import torch.nn.functional as F
import logging
from evaluate import evaluate
import os
import albumentations as A
from albumentations.pytorch import ToTensorV2
from collections import OrderedDict

#from model.unet.unet_model import UNet
#from model.segnet.segnet_model import SegNet
#from torchvision.models.segmentation import deeplabv3_resnet101 as DeepLabv3
from model.ensemblenet_model import EnsembleNet


from utils.dice_score import dice_loss
from utils.data_load import KittiDataset
from torchsummaryX import summary

In [2]:
Val_Percent = 0.3
Scale_Percent = 1.0
Batch_Size = 8
learning_rate = 0.0001
Pin_Memory = False
epochs = 30

#Image_Size = [384, 1242]
Image_Size = [384, 1216]
#Image_Size = [384,384]
Gradient_Clipping = 0.7

#Num_Class = 31
#Num_Class = 21
Num_Class = 2
Num_Channel = 3
amp = True

Model_Name = 'ensemble_voting'


Img_Path =  'data/data_road/training/image_2'
Mask_Path =  'data/data_road/training/semantic'

save_checkpoint = False
checkpoint_dir = '../trained'
batch_size = Batch_Size

In [3]:
dirImg = Path(Img_Path)
dirMask = Path(Mask_Path)

dir_checkpoint = Path(checkpoint_dir)

train_transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        #A.VerticalFlip(p=0.5),
        A.Rotate(limit=30, p=0.5),
        #A.RandomBrightnessContrast(p=0.5),
        #A.RandomGamma(p=0.5),
        #A.RandomSnow(p=0.5),
        #A.RandomRain(p=0.5),
        #A.RandomFog(p=0.5),
        #A.RandomSunFlare(p=0.5),
        A.RandomShadow(p=0.5),
        #A.RandomToneCurve(p=0.5),
        #A.GaussNoise(p=0.5),
        #A.Emboss(p=0.5),  # IAAEmboss 대신 Emboss 사용
        #A.Perspective(p=0.5),  # IAAPerspective 대신 Perspective 사용
        #A.CLAHE(p=0.5)
])

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
datasets =  KittiDataset(dirImg, dirMask, Image_Size, Scale_Percent)
#datasets =  KittiDataset(dirImg, dirMask, Image_Size, Scale_Percent, train_transform)
n_val = int(len(datasets) * Val_Percent)
n_train = len(datasets) - n_val
train_set, val_set = random_split(datasets, [n_train, n_val], generator=torch.Generator().manual_seed(0))

loader_args = dict(batch_size=Batch_Size, num_workers= os.cpu_count(), pin_memory=Pin_Memory)
train_loader = DataLoader(train_set, shuffle=True, drop_last = True, **loader_args)
val_loader = DataLoader(val_set, shuffle=False, drop_last=True, **loader_args)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 289/289 [00:00<00:00, 945.10it/s]


In [6]:
model = EnsembleNet(Model_Name, Num_Channel, Num_Class)
model = model.to(memory_format=torch.channels_last, device = device)

/home/user1/anaconda3/envs/ksh/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/home/user1/anaconda3/envs/ksh/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [7]:
# 4. Set up the optimizer, the loss, the learning rate scheduler and the loss scaling for AMP
#optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=1e-8, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-8)
#optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-8, momentum=0.9)

unet_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2)  # goal: maximize Dice score
segnet_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2)  # goal: maximize Dice score
deeplab_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2)  # goal: maximize Dice score
voting_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2)  # goal: maximize Dice score

grad_scaler = torch.cuda.amp.GradScaler(enabled=amp)
criterion = nn.CrossEntropyLoss()
global_step = 0

In [8]:
def calculate_loss(pred, true_masks, nclass, multiclass):
    loss = criterion(pred, true_masks)
    loss += dice_loss(
        F.softmax(pred, dim=1).float(),
        F.one_hot(true_masks, nclass).permute(0, 3, 1, 2).float(),
        multiclass=multiclass
    )
    return loss


def forward_and_backward(model, images, true_masks, amp, optimizer, grad_scaler, model_name):
    with torch.autocast(device.type if device.type != 'mps' else 'cpu', enabled=amp):
        if model_name == 'ensemble_voting':
            unet_pred, segnet_pred, deeplab_pred = model(images)
            deeplab_pred = deeplab_pred['out']
        else:
            masks_pred = model(images)
            if isinstance(masks_pred, OrderedDict):
                masks_pred = masks_pred['out']

        try:
            mn_cls = model.n_classes
        except:
            mn_cls = model.classifier[-1].out_channels


        if model_name == 'ensemble_voting':
            unet_loss = calculate_loss(unet_pred, true_masks, mn_cls, multiclass=True)
            segnet_loss = calculate_loss(segnet_pred, true_masks, mn_cls, multiclass=True)
            deeplab_loss = calculate_loss(deeplab_pred, true_masks, mn_cls, multiclass=True)
        else:
            squ_masks_pred = masks_pred.squeeze(1) if mn_cls == 1 else masks_pred
            loss = calculate_loss(squ_masks_pred, true_masks, mn_cls, multiclass=False)

    
    optimizer.zero_grad(set_to_none=True)
    if model_name == 'ensemble_voting':
        for pred, loss in zip([unet_pred, segnet_pred, deeplab_pred], [unet_loss, segnet_loss, deeplab_loss]):
            grad_scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), Gradient_Clipping)
            grad_scaler.step(optimizer)
            grad_scaler.update()
            
        return model, unet_loss, segnet_loss, deeplab_loss
    else:
        optimizer.zero_grad(set_to_none=True)
        grad_scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), Gradient_Clipping)
        grad_scaler.step(optimizer)
        grad_scaler.update()
        
        return model, loss


In [9]:
valScore_list = []
TrainLoss_list = []
# 5. Begin training
for epoch in range(1, epochs + 1):
    model.train()
    epoch_loss = 0
    epoch_unet_loss = 0
    epoch_segnet_loss = 0
    epoch_deeplab_loss = 0
    
    with tqdm(total=n_train, desc=f'Epoch {epoch}/{epochs}', unit='img') as pbar:
        for batch in train_loader:
            images, true_masks = batch['image'], batch['mask']

            images = images.to(device=device, dtype=torch.float32, memory_format=torch.channels_last)
            true_masks = true_masks.to(device=device, dtype=torch.long)

                
            result = forward_and_backward(model, images, true_masks, amp, optimizer, grad_scaler, Model_Name)
            
            if len(result) == 4:
                model, unet_loss, segnet_loss, deeplab_loss = result
                
                pbar.update(images.shape[0])
                global_step += 1
                epoch_unet_loss += unet_loss.item()
                epoch_segnet_loss += segnet_loss.item()
                epoch_deeplab_loss += deeplab_loss.item()
                
            elif len(result) == 2:
                model, loss
                pbar.update(images.shape[0])
                global_step += 1
                epoch_loss += loss.item()


                
        if len(result) == 4:
            print('Unet Loss: {}     Segnet Loss: {}     Deelab Loss: {}'.format(unet_loss, segnet_loss, deeplab_loss))
            
        elif len(result) == 2:
            print('{} Loss: {}'.format(Model_Name, loss))


        # Evaluation round
        division_step = (n_train // (5 * batch_size))
        if division_step > 0:
            if global_step % division_step == 0:
                '''
                val_score = evaluate(model, val_loader, device, amp)
                
                scheduler.step(val_score)
                valScore_list.append(val_score)
                TrainLoss_list.append(loss)
                '''
                if len(result) == 4:
                    unet_val_score, segnet_val_score, deeplab_val_score, voting_val_score = evaluate(model, val_loader, device, Model_Name, amp)
                else:
                    val_score = evaluate(model, val_loader, device, Model_Name, amp)
                
                #scheduler.step(val_score)
                
                unet_scheduler.step(unet_val_score)
                segnet_scheduler.step(segnet_val_score)
                deeplab_scheduler.step(deeplab_val_score)
                voting_scheduler.step(deeplab_val_score)
                
                print('Unet Validation Dice score: {}     Segnet Validation Dice score: {}     Deeplab Validation Dice score: {}'.format(unet_val_score, segnet_val_score, deeplab_val_score))
                print('Ensemble Voting Validation Dice score: {} '.format(voting_val_score))
                #valScore_list.append(val_score)
                #TrainLoss_list.append(loss)
                #print('Validation Dice score: {}'.format(val_score))
                                
        
    if save_checkpoint:
        Path(dir_checkpoint).mkdir(parents=True, exist_ok=True)
        torch.save(model.state_dict(), str(dir_checkpoint / 'checkpoint_epoch{}.pth'.format(epoch + 1)))

Epoch 1/20:  99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌   | 200/203 [00:20<00:00, 12.51img/s]

Unet Loss: 0.5431234240531921     Segnet Loss: 0.7385768890380859     Deelab Loss: 0.3700820207595825


Epoch 1/20:  99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌   | 200/203 [00:24<00:00,  8.29img/s]


Unet Validation Dice score: 0.4861016273498535     Segnet Validation Dice score: 0.01578707806766033     Deeplab Validation Dice score: 0.7020008563995361
Ensemble Voting Validation Dice score: 0.6945091485977173 


Epoch 2/20:  99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌   | 200/203 [00:17<00:00, 12.37img/s]

...

Unet Validation Dice score: 0.9075689315795898     Segnet Validation Dice score: 0.8948413133621216     Deeplab Validation Dice score: 0.9490095376968384
Ensemble Voting Validation Dice score: 0.9241560101509094 


Epoch 20/20:  99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌   | 200/203 [00:17<00:00, 12.32img/s]

Unet Loss: 0.14293953776359558     Segnet Loss: 0.07232735306024551     Deelab Loss: 0.030523674562573433


Epoch 20/20:  99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌   | 200/203 [00:21<00:00,  9.25img/s]

Unet Validation Dice score: 0.9088501334190369     Segnet Validation Dice score: 0.9008020758628845     Deeplab Validation Dice score: 0.9496346712112427
Ensemble Voting Validation Dice score: 0.9270095229148865 


In [16]:
def train_model(__model__, __images__, __true_masks__, __amp__, model_name):
    
        
        if model_name != 'ensemble_voting':    
            with torch.autocast(device.type if device.type != 'mps' else 'cpu', enabled=__amp__):
                masks_pred = __model__(__images__)
                if isinstance(masks_pred, OrderedDict):
                    masks_pred = masks_pred['out']

                try:
                    mn_cls = __model__.n_classes
                except:
                    mn_cls = __model__.classifier[-1].out_channels


                if mn_cls == 1:

                    squ_masks_pred = masks_pred.squeeze(1)
                    loss = criterion(squ_masks_pred, __true_masks__.float())
                    loss += dice_loss(F.sigmoid(squ_masks_pred), __true_masks__.float(), multiclass=False)


                else:
                    loss = criterion(masks_pred, __true_masks__)
                    loss += dice_loss(
                        F.softmax(masks_pred, dim=1).float(),
                        F.one_hot(__true_masks__, mn_cls).permute(0, 3, 1, 2).float(),
                        multiclass=True
                    )

            optimizer.zero_grad(set_to_none=True)
            grad_scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(__model__.parameters(), Gradient_Clipping)
            grad_scaler.step(optimizer)
            grad_scaler.update()

            return model, loss

        elif model_name == 'ensemble_voting':
            with torch.autocast(device.type if device.type != 'mps' else 'cpu', enabled=__amp__):
                
                __unet_pred__, __segnet_pred__, __deeplab_pred__ = __model__(__images__)
                __deeplab_pred__ = __deeplab_pred__['out']
                
                mn_cls = __model__.n_classes
                

                __unet_loss__ = criterion(__unet_pred__, __true_masks__)
                __unet_loss__ += dice_loss(
                        F.softmax(__unet_pred__, dim=1).float(),
                        F.one_hot(__true_masks__, mn_cls).permute(0, 3, 1, 2).float(),
                        multiclass=True )        

                __segnet_loss__ = criterion(__segnet_pred__, __true_masks__)
                __segnet_loss__ += dice_loss(
                        F.softmax(__segnet_pred__, dim=1).float(),
                        F.one_hot(__true_masks__, mn_cls).permute(0, 3, 1, 2).float(),
                        multiclass=True )

                __deeplab_loss__ = criterion(__deeplab_pred__, __true_masks__)
                __deeplab_loss__ += dice_loss(
                            F.softmax(__deeplab_pred__, dim=1).float(),
                            F.one_hot(__true_masks__, mn_cls).permute(0, 3, 1, 2).float(),
                            multiclass=True )
                
            optimizer.zero_grad(set_to_none=True)
            grad_scaler.scale(__unet_loss__).backward()
            torch.nn.utils.clip_grad_norm_(__model__.parameters(), Gradient_Clipping)
            grad_scaler.step(optimizer)
            grad_scaler.update()   

            optimizer.zero_grad(set_to_none=True)
            grad_scaler.scale(__segnet_loss__).backward()
            torch.nn.utils.clip_grad_norm_(__model__.parameters(), Gradient_Clipping)
            grad_scaler.step(optimizer)
            grad_scaler.update() 
            
            optimizer.zero_grad(set_to_none=True)
            grad_scaler.scale(__deeplab_loss__).backward()
            torch.nn.utils.clip_grad_norm_(__model__.parameters(), Gradient_Clipping)
            grad_scaler.step(optimizer)
            grad_scaler.update()   

            return __unet_pred__, __segnet_pred__, __deeplab_pred__, __unet_loss__, __segnet_loss__, __deeplab_loss__

In [18]:
valScore_list = []
TrainLoss_list = []
# 5. Begin training
for epoch in range(1, epochs + 1):
    model.train()
    epoch_loss = 0
    with tqdm(total=n_train, desc=f'Epoch {epoch}/{epochs}', unit='img') as pbar:
        for batch in train_loader:
            images, true_masks = batch['image'], batch['mask']

            images = images.to(device=device, dtype=torch.float32, memory_format=torch.channels_last)
            true_masks = true_masks.to(device=device, dtype=torch.long)
            '''
            with torch.autocast(device.type if device.type != 'mps' else 'cpu', enabled=amp):
                
                masks_pred = model(images)
                if isinstance(masks_pred, OrderedDict):
                    masks_pred = masks_pred['out']
                             
                try:
                    mn_cls = model.n_classes
                except:
                    mn_cls = model.classifier[-1].out_channels
                    
                
                if mn_cls == 1:
                    
                    squ_masks_pred = masks_pred.squeeze(1)
                    loss = criterion(squ_masks_pred, true_masks.float())
                    loss += dice_loss(F.sigmoid(squ_masks_pred), true_masks.float(), multiclass=False)
                    
                    #loss = criterion(masks_pred.squeeze(1), true_masks.float())
                    #loss += dice_loss(masks_pred, true_masks, multiclass=False)
                    
                else:
                    loss = criterion(masks_pred, true_masks)
                    loss += dice_loss(
                        F.softmax(masks_pred, dim=1).float(),
                        F.one_hot(true_masks, mn_cls).permute(0, 3, 1, 2).float(),
                        multiclass=True
                    )
                

            optimizer.zero_grad(set_to_none=True)
            grad_scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), Gradient_Clipping)
            grad_scaler.step(optimizer)
            grad_scaler.update()
            '''
            
            
            
            #model, loss = train_model(model, images, true_masks, amp, Model_Name)
            
            unet_model, segnet_model, deeplab_model, unet_loss, segnet_loss, deeplab_loss = train_model(model, images, true_masks, amp, Model_Name)
                
            
            
            pbar.update(images.shape[0])
            global_step += 1
            #epoch_loss += loss.item()
            
        print('Training unet Loss: {}'.format(unet_loss))
        print('Training segnet Loss: {}'.format(segnet_loss))
        print('Training deelab Loss: {}'.format(deeplab_loss))
            
        '''
            # Evaluation round
            division_step = (n_train // (5 * batch_size))
            if division_step > 0:
                if global_step % division_step == 0:

                    val_score = evaluate(model, val_loader, device, amp)
                    
                    scheduler.step(val_score)
                    valScore_list.append(val_score)
                    TrainLoss_list.append(loss)
                    

                    #logging.info('Validation Dice score: {}'.format(val_score))
                    print('Training Dice Loss: {}'.format(loss))
                    print('Validation Dice score: {}'.format(val_score))
                                
        '''
    if save_checkpoint:
        Path(dir_checkpoint).mkdir(parents=True, exist_ok=True)
        torch.save(model.state_dict(), str(dir_checkpoint / 'checkpoint_epoch{}.pth'.format(epoch + 1)))

Epoch 1/10:  99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌   | 200/203 [00:19<00:00, 10.26img/s]


Training unet Loss: 0.39075762033462524
Training segnet Loss: 0.7669399976730347
Training deelab Loss: 0.4201391637325287


Epoch 2/10:  99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌   | 200/203 [00:16<00:00, 11.77img/s]


Training unet Loss: 0.46772128343582153
Training segnet Loss: 0.4714847207069397
Training deelab Loss: 0.25792646408081055


Epoch 3/10:  99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌   | 200/203 [00:17<00:00, 11.59img/s]


...

Training unet Loss: 0.2397838532924652
Training segnet Loss: 0.21610693633556366
Training deelab Loss: 0.04417917877435684


Epoch 9/10:  99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌   | 200/203 [00:17<00:00, 11.61img/s]


Training unet Loss: 0.1764993965625763
Training segnet Loss: 0.13347478210926056
Training deelab Loss: 0.035884156823158264


Epoch 10/10:  99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌   | 200/203 [00:17<00:00, 11.50img/s]

Training unet Loss: 0.15233871340751648
Training segnet Loss: 0.11093688011169434
Training deelab Loss: 0.0344683937728405


In [11]:
valScore_list

[tensor(0.4461, device='cuda:0'),
 tensor(0.7066, device='cuda:0'),
 tensor(0.7842, device='cuda:0'),
 tensor(0.7259, device='cuda:0'),
 tensor(0.7876, device='cuda:0'),
 tensor(0.8873, device='cuda:0'),
 tensor(0.9105, device='cuda:0'),
 tensor(0.8723, device='cuda:0'),
 tensor(0.9304, device='cuda:0'),
 tensor(0.9438, device='cuda:0'),
 tensor(0.9486, device='cuda:0'),
 tensor(0.9297, device='cuda:0'),
 tensor(0.9496, device='cuda:0'),
 tensor(0.9369, device='cuda:0'),
 tensor(0.9490, device='cuda:0'),
 tensor(0.9288, device='cuda:0'),
 tensor(0.9442, device='cuda:0'),
 tensor(0.9550, device='cuda:0'),
 tensor(0.9571, device='cuda:0'),
 tensor(0.9579, device='cuda:0'),
 tensor(0.9598, device='cuda:0'),
 tensor(0.9610, device='cuda:0'),
 tensor(0.9620, device='cuda:0'),
 tensor(0.9627, device='cuda:0'),
 tensor(0.9636, device='cuda:0'),
 tensor(0.9634, device='cuda:0'),
 tensor(0.9643, device='cuda:0'),
 tensor(0.9641, device='cuda:0'),
 tensor(0.9634, device='cuda:0'),
 tensor(0.9640

In [12]:
TrainLoss_list

[tensor(0.3917, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.1367, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.1151, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.2557, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.1054, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.0902, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.0724, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.0550, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.0648, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.0617, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.0639, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.0401, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.0525, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.0346, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.0353, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.0692, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.0477, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.0306